# Benutzung des Tools

1. Unter dem Namen der Datei auf Runtime/Laufzeit drücken.
2. Run all auswählen.
3. Auf Eingabenabfrage warten und diese tätigen.

In [1]:
import pandas_datareader.data as reader
import pandas as pd
import numpy as np
import datetime as dt
import math

## Eingabeaufforderung

Hier die angeforderten Eingaben ausfüllen.

- Die Anzahl der enthaltenen Positionen im Portfolio
- Die Position und die monatliche Sparrate dieser Position
- Das Startdatum des Portfolios

Es lassen sich mit diesem Tool nicht nur ETFs und Aktien, sondern auch Rohstoffe und alle Daten, die über Yahoo Finance zu finden sind analysieren.

In [2]:
## Erstellen des Portfolios und Zeitraumes
num_assets = int(input("Wieviele Assets sind in Ihrem Musterportfolio? "))

asset_list = []
rates_list = []
i = 0
for i in range(num_assets):
  asset_list.append(input("Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? "))
  rates_list.append(float(input("Wieviel zahlen monatlich in dieses Asset? ")))

portfolio = dict(zip(asset_list, rates_list))

formation = input("Bitte geben Sie den Beginn des Portfolios ein (Format: 2015,12,7): ").split(",")
formation = dt.datetime(int(formation[0]), int(formation[1]), int(formation[2]))

Wieviele Assets sind in Ihrem Musterportfolio? 5
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? AAPL
Wieviel zahlen monatlich in dieses Asset? 100
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? TSLA
Wieviel zahlen monatlich in dieses Asset? 100
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? SAP.DE
Wieviel zahlen monatlich in dieses Asset? 100
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? O
Wieviel zahlen monatlich in dieses Asset? 100
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? FB
Wieviel zahlen monatlich in dieses Asset? 100
Bitte geben Sie den Beginn des Portfolios ein (Format: 2015,12,7): 2015,1,1


In [3]:
## Schlusskurse und Veränderungen ermitteln

df = reader.get_data_yahoo(asset_list,formation,interval='m')['Adj Close']
mtl_ret = df.pct_change() + 1
mtl_ret = mtl_ret.reset_index(drop=True)

In [10]:
# DataFrame mit den Startwerten
portfolio_werte = pd.DataFrame(columns=asset_list, index=[0])
for asset in asset_list:
  portfolio_werte.loc[0,asset] = portfolio[asset]


# Berechnung der Endwerte für die einzelnen Assets
for i in range(1, mtl_ret.shape[0]):
  for asset in asset_list:
    portfolio_werte[asset] += portfolio[asset]
    if not math.isnan(mtl_ret.loc[i, asset]):
      portfolio_werte[asset] *= mtl_ret.loc[i,asset]

In [11]:
# Referenzwerte berechnen
referenz_list = ['URTH', '^GSPC', '^GDAXI']
dfref = reader.get_data_yahoo(referenz_list, formation,interval='m')['Adj Close']
mtl_retref = dfref.pct_change() + 1
mtl_retref = mtl_retref.reset_index(drop=True)

monatliche_einz = 0
for i in rates_list:
  monatliche_einz += i

referenz_werte = pd.DataFrame({'URTH': monatliche_einz,'^GSPC': monatliche_einz,'^GDAXI':monatliche_einz}, index=[0])

for x in range(1, mtl_retref.shape[0]):
  for referenz in referenz_list:
    referenz_werte[referenz] += monatliche_einz
    if not math.isnan(mtl_retref.loc[x,referenz]):
      referenz_werte[referenz] *= mtl_retref.loc[x,referenz]

In [12]:
gesamt_depotwert = 0
for column in portfolio_werte:
  gesamt_depotwert += round(portfolio_werte[column][0],2)
  print(f"Ihr {column} wäre auf einen Wert von {round(portfolio_werte[column][0],2)}€ gewachsen.")

print(f"Ihr gesamtes Portfolio wäre somit auf einen Wert von {round(gesamt_depotwert,2)}€ gewachsen.")
print("")
print("Zum Vergleich ein paar Indizes:")


for x in referenz_werte:
  print(f"Ein {x} wäre bei gleicher Sparrate einen Wert von {round(referenz_werte[x][0],2)}€ gewachsen.")


Ihr AAPL wäre auf einen Wert von 54029.52€ gewachsen.
Ihr TSLA wäre auf einen Wert von 210142.72€ gewachsen.
Ihr SAP.DE wäre auf einen Wert von 18610.86€ gewachsen.
Ihr O wäre auf einen Wert von 17380.16€ gewachsen.
Ihr FB wäre auf einen Wert von 28511.36€ gewachsen.
Ihr gesamtes Portfolio wäre somit auf einen Wert von 328674.62€ gewachsen.

Zum Vergleich ein paar Indizes:
Ein URTH wäre bei gleicher Sparrate einen Wert von 111280.67€ gewachsen.
Ein ^GSPC wäre bei gleicher Sparrate einen Wert von 112642.85€ gewachsen.
Ein ^GDAXI wäre bei gleicher Sparrate einen Wert von 87631.25€ gewachsen.
